In [1]:
pip install streamlit


In [2]:
pip install --upgrade bokeh

In [4]:
import pandas as pd
from bokeh.plotting import figure, curdoc
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.models import Slider, Select
from bokeh.palettes import Spectral6
from bokeh.layouts import column, row

# Load the data
data = pd.read_csv('https://github.com/iqqy19/Visdat/blob/main/Data_Tanaman_Padi_Sumatera_version_1.csv?raw=true')
data.set_index('Tahun', inplace=True)

# Get unique list of provinces
prov_list = data.Provinsi.unique().tolist()

# Make a color mapper
color_mapper = CategoricalColorMapper(factors=prov_list, palette=Spectral6)

# Rename columns
data.rename(columns={'Suhu rata-rata':'suhu_rata','Curah hujan':'curah_hujan','Luas Panen':'luas_panen'}, inplace=True)

# Create the initial ColumnDataSource
source = ColumnDataSource(data={
    'x': data.loc[1993].Produksi,
    'y': data.loc[1993].luas_panen,
    'provinsi': data.loc[1993].Provinsi,
})

# Create the initial figure
plot = figure(title='1993', x_axis_label='Jumlah Produksi', y_axis_label='Luas Panen',
              height=400, width=700, tools=[HoverTool(tooltips='@provinsi')])

# Create the scatter plot
plot.circle(x='x', y='y', source=source, fill_alpha=0.8,
            color=dict(field='provinsi', transform=color_mapper), legend_field='provinsi')

# Set the legend location
plot.legend.location = 'top_right'

# Create the slider widget
slider = Slider(start=1993, end=2020, value=1993, step=1, title='Year')

# Create the dropdown select widgets
x_select = Select(options=['Produksi', 'luas_panen', 'curah_hujan', 'Kelembapan', 'suhu_rata'],
                  value='Produksi', title='x-axis data')
y_select = Select(options=['Produksi', 'luas_panen', 'curah_hujan', 'Kelembapan', 'suhu_rata'],
                  value='luas_panen', title='y-axis data')

# Callback function to update plot
def update_plot(attr, old, new):
    yr = slider.value
    x = x_select.value
    y = y_select.value

    plot.xaxis.axis_label = x
    plot.yaxis.axis_label = y

    new_data = {
        'x': data.loc[yr][x],
        'y': data.loc[yr][y],
        'provinsi': data.loc[yr]['Provinsi']
    }

    source.data = new_data
    plot.title.text = 'Data Tanaman Padi Sumatera %d' % yr

# Attach the callback to the widgets
slider.on_change('value', update_plot)
x_select.on_change('value', update_plot)
y_select.on_change('value', update_plot)

# Create the layout
layout = column(row(slider, x_select, y_select), plot)

# Add the layout to the current document
curdoc().add_root(layout)
